# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [11]:
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score,recall_score,precision_score,classification_report

In [3]:
# load data from database
engine = create_engine('sqlite:///amars_disaster.db')
df = pd.read_sql_table('etl_pipeline',engine)
X = df['message']
Y = df.iloc[:,4:]

In [4]:
X.head(2),Y.head(2)

(0    Weather update - a cold front from Cuba that c...
 1              Is the Hurricane over or is it not over
 Name: message, dtype: object,
    related  request  offer  aid_related  medical_help  medical_products  \
 0        1        0      0            0             0                 0   
 1        1        0      0            1             0                 0   
 
    search_and_rescue  security  military  child_alone      ...        \
 0                  0         0         0            0      ...         
 1                  0         0         0            0      ...         
 
    aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
 0            0                     0                0       0      0     0   
 1            0                     0                1       0      1     0   
 
    earthquake  cold  other_weather  direct_report  
 0           0     0              0              0  
 1           0     0              0              0  
 
 [2 ro

### 2. Write a tokenization function to process your text data

In [5]:

def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [35]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:

X_train, X_test, y_train, y_test = train_test_split(X, Y,random_state=42)


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)



In [16]:
y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7917,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25322,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22191,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
18442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
y_pred_df = pd.DataFrame(y_pred,columns=y_test.columns)

In [29]:
y_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [31]:
for column_name in y_test.columns:
    print(classification_report(y_pred_df[column_name],y_test[column_name]))

             precision    recall  f1-score   support

          0       0.36      0.60      0.45       930
          1       0.93      0.81      0.87      5620
          2       0.06      0.75      0.12         4

avg / total       0.85      0.78      0.81      6554

             precision    recall  f1-score   support

          0       0.98      0.88      0.93      6092
          1       0.34      0.82      0.48       462

avg / total       0.94      0.88      0.90      6554

             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      0.99      1.00      6554

             precision    recall  f1-score   support

          0       0.87      0.72      0.79      4698
          1       0.51      0.74      0.60      1856

avg / total       0.77      0.73      0.74      6554

             precision    recall  f1-score   support

          0       1.00      0.92 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [40]:
parameters = {'features__text_pipeline__vect__max_df':[0.5,1.0],'features__text_pipeline__vect__max_features': (None, 5000),
    'features__text_pipeline__tfidf__use_idf': (True, False)
}

# create grid search object
cv = GridSearchCV(pipeline,param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [41]:
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__max_df': [0.5, 1.0], 'features__text_pipeline__vect__max_features': (None, 5000), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
cv.best_params_

{'features__text_pipeline__tfidf__use_idf': False,
 'features__text_pipeline__vect__max_df': 0.5,
 'features__text_pipeline__vect__max_features': 5000}

In [42]:
y_pred2  = cv.predict(X_test)
y_pred2_df = pd.DataFrame(y_pred2,columns=y_test.columns)
for column_name in y_test.columns:
    print(classification_report(y_pred2_df[column_name],y_test[column_name]))

             precision    recall  f1-score   support

          0       0.39      0.60      0.47      1015
          1       0.92      0.82      0.87      5511
          2       0.34      0.57      0.43        28

avg / total       0.83      0.79      0.81      6554

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      6007
          1       0.39      0.79      0.52       547

avg / total       0.93      0.88      0.90      6554

             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      0.99      1.00      6554

             precision    recall  f1-score   support

          0       0.85      0.74      0.79      4472
          1       0.57      0.73      0.64      2082

avg / total       0.76      0.74      0.74      6554

             precision    recall  f1-score   support

          0       0.99      0.93 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.